In [86]:
import pandas as pd

In [87]:
df = pd.read_json('dataset_incidenti.json')
#elimina colonne inutili, aggrega numero di incidenti, feriti, morti, segnaletica, particolarita strada (ordina per data e prendi il primo (più recente))


In [88]:
def to_float(value):
    if type(value)==float:
        return value
    else:
        return float(value.replace(',', '.'))
df['Latitudine'] = df['Latitudine'].apply(to_float)
df['Longitudine'] = df['Longitudine'].apply(to_float)

In [89]:
print(df['Latitudine'].apply(type).value_counts())
print(df['Longitudine'].apply(type).value_counts())


Latitudine
<class 'float'>    223984
Name: count, dtype: int64
Longitudine
<class 'float'>    223984
Name: count, dtype: int64


In [90]:
# Conta il numero di volte che una stessa strada si ripete nel DataFrame
conteggio_strade = df['Strada'].value_counts().reset_index()

# Rinomina le colonne risultanti
conteggio_strade.columns = ['Strada', 'Numero_Incidenti']

# Unisci il conteggio delle strade al DataFrame originale
df = pd.merge(df, conteggio_strade, on='Strada', how='left')

print(df)

              Data Localizzazione                 Strada  \
0       13/07/2007  Strada Urbana        VIALE CASTRENSE   
1       05/12/2008  Strada Urbana   PIAZZA MASSA CARRARA   
2       31/08/2009  Strada Urbana             VIA CAVOUR   
3       23/01/2010  Strada Urbana           VIA MERULANA   
4       22/03/2010  Strada Urbana        VIA DEGLI ACERI   
...            ...            ...                    ...   
223979  27/12/2021  Strada Urbana        PONTE GARIBALDI   
223980  02/11/2021  Strada Urbana              VIA TEANO   
223981  29/12/2021  Strada Urbana  VIALE DELLA PRIMAVERA   
223982  22/09/2021  Strada Urbana  VIA BIORDO MICHELOTTI   
223983  21/12/2021  Strada Urbana      VIA DEL TORRACCIO   

                                          NaturaIncidente  \
0                  Scontro laterale fra veicoli in marcia   
1                                  Investimento di pedone   
2       Scontro frontale/laterale SX fra veicoli in ma...   
3                  Infortunio per s

In [92]:
df_compattato = df.groupby(['Strada']).agg({
    'Localizzazione': 'first',
    'ParticolaritaStrade': 'first',
    'Segnaletica': 'first',
    'Latitudine': ['min', 'max'],
    'Longitudine': ['min', 'max'],
    'Feriti': 'sum',
    'Deceduti': 'sum',
    'Numero_Incidenti': 'first'
}).reset_index()

df_compattato.columns = ['Strada', 'Localizzazione', 'ParticolaritaStrade', 'Segnaletica',
                  'Min_Latitudine', 'Max_Latitudine', 'Min_Longitudine', 'Max_Longitudine',
                  'Totale_Feriti', 'Totale_Deceduti', 'Numero_Incidenti']

print(df_compattato)


                                            Strada        Localizzazione  \
0                              Sottovia Cappellini         Strada Urbana   
1          AEROPORTO CIAMPINO - LARGO C. D'ASCANIO  Comunale extraurbana   
2     AEROPORTO CIAMPINO - LARGO LEONARDO DA VINCI          Altra strada   
3            AEROPORTO CIAMPINO - VIA DE BERNARDIS          Altra strada   
4        AEROPORTO CIAMPINO - VIA FRANCESCO AGELLO          Altra strada   
...                                            ...                   ...   
9165                       Via della Stella Polare         Strada Urbana   
9166                                     tang. Est         Strada Urbana   
9167                tangenziale / est dir. salaria         Strada Urbana   
9168                           via del Colle Oppio         Strada Urbana   
9169                              via g. zirardini         Strada Urbana   

     ParticolaritaStrade               Segnaletica  Min_Latitudine  \
0               I

In [93]:
df_compattato = df_compattato.rename(columns={"Numero_Incidenti": "NumeroIncidenti"})

In [94]:
print(df_compattato.columns)

Index(['Strada', 'Localizzazione', 'ParticolaritaStrade', 'Segnaletica',
       'Min_Latitudine', 'Max_Latitudine', 'Min_Longitudine',
       'Max_Longitudine', 'Totale_Feriti', 'Totale_Deceduti',
       'NumeroIncidenti'],
      dtype='object')


In [96]:
df_compattato.to_json("dataset_strade.json")